In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import time
import numpy as np
import cifar10, cifar10_input
 
#define patameters
max_steps = 100

batch_size = 512
n_fc_1 = 128
n_fc_2 = 64
display_step = 200
data_dir = 'cifar10_data/cifar-10-batches-bin/'
 
#define L2 regularization for weight loss
def l2_weight_loss(shape, stddev, w_1):
    '''
description: to adopt L2 regularization for weight to prevent over-fitting
Args:	shape:
stddev: standard deviation
w_1: weight coeffient
Returns:
weight: the regularized weight coeffient
'''
    weight = tf.Variable(tf.truncated_normal(shape, stddev))
    if w_1 is not None:
        weight_loss = tf.multiply(tf.nn.l2_loss(weight), w_1, name='weight_loss')
        tf.add_to_collection('losses', weight_loss)
    return weight
 
#define weight initializer
def weight_init(shape, stddev):
    return tf.Variable(tf.truncated_normal(shape, stddev))
 
#define biases initializer
def biases_init(shape):

    return tf.Variable(tf.random_normal(shape))
 
#define conv layer
def conv2d(x_image, weight):

    return tf.nn.conv2d(x_image, weight, strides = [1, 1, 1, 1], padding = 'SAME')
 
#define max pooling layer
def max_pool(x_image):

    return tf.nn.max_pool(x_image, ksize = [1, 3, 3, 1], strides = [1, 2, 2, 1], padding = 'SAME')
 
def lrn_norm(x_image):

    return tf.nn.lrn(x_image, 4, bias = 1.0, alpha = 0.001 / 9.0, beta = 0.75)
 
#image enhancement processing for trainning data sets
train_images, train_labels = cifar10_input.distorted_inputs(batch_size = batch_size, data_dir = data_dir)
 
#define input placeholder
x_images = tf.placeholder(tf.float32, [batch_size, 24, 24, 3],name='x_images')
x_labels = tf.placeholder(tf.float32, [batch_size],name='x_labels')
keep_prob = tf.placeholder("float",name='keep_prob') 
#define layer 1
w_1 = weight_init([5, 5, 3, 32], 0.05)
b_1 = biases_init([32])
conv_1 = tf.nn.relu(conv2d(x_images, w_1) + b_1)
pool_1 = max_pool(conv_1)
lrn_1 = lrn_norm(pool_1)
 
#define layer 2
w_2 = weight_init([5, 5, 32, 32], 0.05)
b_2 = biases_init([32])
conv_2 = tf.nn.relu(conv2d(lrn_1, w_2) + b_2)
pool_2 = max_pool(conv_2)
 
#define flatten layer
re_shape = tf.reshape(pool_2, [batch_size, -1])
n_input = re_shape.get_shape()[1].value
 
#define full connection 1
w_3 = l2_weight_loss([n_input, n_fc_1], 0.05, w_1 = 0.001)
b_3 = biases_init([n_fc_1])
fc_1 = tf.nn.relu(tf.matmul(re_shape, w_3) + b_3)
 
#define full connection 2
w_4 = l2_weight_loss([n_fc_1, n_fc_2], 0.05, w_1 = 0.003)
b_4 = biases_init([n_fc_2])
fc_2 = tf.nn.relu(tf.matmul(fc_1, w_4) + b_4)
 
#define output layer
w_5 = weight_init([n_fc_2, 10], 1.0 / 96.0)
b_5 = biases_init([10])
logits = tf.add(tf.matmul(fc_2, w_5), b_5)
y_pred = tf.nn.softmax(logits,name='y_pred')
 
#define loss function
x_labels = tf.cast(x_labels, tf.int32)
cross_enerty = tf.nn.sparse_softmax_cross_entropy_with_logits(logits = logits, labels = x_labels, name = 'cross_enerty_pre_example')
losses = tf.reduce_mean(cross_enerty, name = 'cross_enerty')
tf.add_to_collection('losses', losses)
loss = tf.add_n(tf.get_collection('losses'), name = 'total_loss')
 
#define optimizer
optimizer = tf.train.AdamOptimizer(learning_rate = 0.0001).minimize(loss)
saver = tf.train.Saver() #定义saver
#display test result
test_images, test_labels = cifar10_input.inputs(batch_size = batch_size, data_dir = data_dir)
 
#claculate trainning accuracy
def accuracy(test_labels, y_pred):

    test_labels = tf.to_int64(test_labels)
    correction_pred = tf.equal(test_labels, tf.argmax(y_pred, 1))
    acc = tf.reduce_mean(tf.cast(correction_pred, tf.float32))
    return acc
 
#create session
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
 
#create thread to accelaeate processing efficience
tf.train.start_queue_runners(sess = sess)
 
#start trainning and display
cross_loss = []
start_time = time.time()
for i in range(max_steps):

    batch_xs, batch_ys = sess.run([train_images, train_labels])
#     print(batch_xs,batch_ys)
    _, c = sess.run([optimizer, loss], feed_dict = {x_images:batch_xs, x_labels:batch_ys})
    cross_loss.append(c)
    every_epoch_time = time.time() - start_time
    if i % display_step == 0:
        examples_per_sec = batch_size / every_epoch_time
        every_batch_time = float(every_epoch_time)

        print('Epoch : ', '%d'%(i+100), 'loss : ', '{:.5f}'.format(c))
        
cost_time = time.time() - start_time

saver.save(sess, '/tmp/tf_cifar_cnn2/model.ckpt') #模型储存位置


print("optimization finished ! 总耗时",+str(cost_time))
 
#display loss processing
# fig, ax = plt.subplots(13, 6)
# ax.plot(cross_loss)
# plt.grid()
# plt.title("trian loss")
# plt.show() 
 
#claculate test accuracy
for i in range(10):
    test_acc = []
    batch_xs, batch_ys = sess.run([test_images, test_labels])
    batch_y_pred = sess.run(y_pred, feed_dict = {x_images:batch_xs})
#     print('这是batch_y_pred',batch_y_pred)
#     print('这是batch_ys',batch_ys)


    test_accuracy = accuracy(batch_ys, batch_y_pred)
    
    acc = sess.run(test_accuracy, feed_dict = {x_images:batch_xs})
#     train_accuracy = acc.eval(feed_dict={x_images:batch_xs, y_pred: batch_ys,keep_prob: 1.0},session=sess)
    test_acc.append(acc)
    print("test accuracy : ", acc)
#     print("test accuracy : ", acc,'train_accuracy',train_accuracy)

print("mean accuracy : ", np.mean(test_acc))

/usr/local/lib/python3.6/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)



Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
这是filename_queue <tensorflow.python.ops.data_flow_ops.FIFOQueue object at 0x7fc823232e48>
Instructions for updating:
Queue-based input pipelines have been 

TypeError: bad operand type for unary +: 'str'

In [ ]:
# #claculate trainning accuracy
# def accuracy(test_labels, y_pred):

#     test_labels = tf.to_int64(test_labels)
#     correction_pred = tf.equal(test_labels, tf.argmax(y_pred, 1))
#     acc = tf.reduce_mean(tf.cast(correction_pred, tf.float32))
#     return acc
 
# #claculate test accuracy
# for i in range(10):
#     test_acc = []
#     batch_xs, batch_ys = sess.run([test_images, test_labels])
#     batch_y_pred = sess.run(y_pred, feed_dict = {x_images:batch_xs})
#     acc = accuracy(batch_ys, batch_y_pred)
#     test_acc.append(acc)
#     print("test accuracy : ", acc)
# print("mean accuracy : ", np.mean(test_acc))



这是batch_y_pred [[0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
这是batch_ys [3 8 8 0 6 6 1 6 3 1 0 9 5 7 9 8 5 7 8 6 7 0 4 9 5 2 4 0 9 6 6 5 4 5 9 2 4
 1 9 5 4 6 5 6 0 9 3 9 7 6 9 8 0 3 8 8 7 7 4 6 7 3 6 3 6 2 1 2 3 7 2 6 8 8
 0 2 9 3 3 8 8 1 1 7 2 5 2 7 8 9 0 3 8 6 4 6 6 0 0 7 4 5 6 3 1 1 3 6 8 7 4
 0 6 2 1 3 0 4 2 7 8 3 1 2 8 0 8 3 5 2 4 1 8 9 1 2 9 7 2 9 6 5 6 3 8 7 6 2
 5 2 8 9 6 0 0 5 2 9 5 4 2 1 6 6 8 4 8 4 5 0 9 9 9 8 9 9 3 7 5 0 0 5 2 2 3
 8 6 3 4 0 5 8 0 1 7 2 8 8 7 8 5 1 8 7 1 3 0 5 7 9 7 4 5 9 8 0 7 9 8 2 7 6
 9 4 3 9 6 4 7 6 5 1 5 8 8 0 4 0 5 5 1 1 8 9 0 3 1 9 2 2 5 3 9 9 4 0 3 0 0
 9 8 1 5 7 0 8 2 4 7 0 2 3 6 3 8 5 0 3 4 3 9 0 6 1 0 9 1 0 7 9 1 2 6 9 3 4
 6 0 0 6 6 6 3 2 6 1 8 2 1 6 8 6 8 0 4 0 7 7 5 5 3 5 2 3 4 1 7 5 4 6 1 9 3
 6 6 9 3 8 0 7 2 6 2 5 8 5 4 6 8 9 9 1 0 2 2 7 3 2 8 0 9 5 8 1 9 4 1 3 8 1
 4 7 9 4 2 7 0 7 0 6 6 9 0 9 2 8 7 2 2 5 1 2 6 2 9 6 2 3 0 3 9 8 7 8 8 4 0
 1 8 2 7 9 3 6 1 9 0 7 3 7 4 5 0 0 2 9 3 4 0 6 2 5 3 7 3 7 2 5 3 1 1 4 9 9
 5 7 5 0 2 2 2 9 7 3 9 4 3 5 4 6 5 6 1 4 3 4 4 3 7 8 3 7 8 0 5 7 6 0 5 4 8
 6 8 5 5 9 9 9 5 0 1 0 8 1 1 8 0 2 2 0 4 6 5 4 9 4 7 9 9 4 5 6]